In [50]:
import json
import pandas as pd
import plotly.express as px

In [113]:
test_data2 = pd.read_csv('time_tweet.csv').drop(['Text'], axis=1)
print(test_data2.groupby(['User', 'Sentiment']).count())
test_data2.groupby(['User', 'Sentiment']).count().unstack()

                          Time
User           Sentiment      
Aquinas_Quotes Negative      3
               Neutral      12
               Positive      3
ArchCordileone Negative      1
               Neutral       4
...                        ...
stlaurencecha  Neutral      15
               Positive      3
timotheeology  Negative      5
               Neutral      12
               Positive      2

[160 rows x 1 columns]


Time                 
Sentiment       Negative Neutral Positive
User                                     
Aquinas_Quotes       3.0    12.0      3.0
ArchCordileone       1.0     4.0      5.0
BVMConsolatrix      19.0    29.0      2.0
BishopAthanas1       NaN     1.0      NaN
Bishop_Sheen         2.0     1.0      1.0
...                  ...     ...      ...
settlerslament       9.0    19.0      4.0
sistervpaul_         NaN    11.0      5.0
staidansheritag      NaN     5.0      2.0
stlaurencecha        NaN    15.0      3.0
timotheeology        5.0    12.0      2.0

[64 rows x 3 columns]

In [117]:
test_data = pd.read_csv('time_tweet.csv').drop(['Text'], axis=1)
pt = pd.pivot_table(test_data, values='Time', index='User', columns='Sentiment', aggfunc='count').fillna(0)
# pt.iloc[0].name
# pt['Username'] = pt.index
# pt.set_index('Username')
# pt.index
pt.reset_index()

Sentiment,User,Negative,Neutral,Positive
0,Aquinas_Quotes,3.0,12.0,3.0
1,ArchCordileone,1.0,4.0,5.0
2,BVMConsolatrix,19.0,29.0,2.0
3,BishopAthanas1,0.0,1.0,0.0
4,Bishop_Sheen,2.0,1.0,1.0
...,...,...,...,...
59,settlerslament,9.0,19.0,4.0
60,sistervpaul_,0.0,11.0,5.0
61,staidansheritag,0.0,5.0,2.0
62,stlaurencecha,0.0,15.0,3.0


In [92]:
pt['User']

User
Aquinas_Quotes      Aquinas_Quotes
ArchCordileone      ArchCordileone
BVMConsolatrix      BVMConsolatrix
BishopAthanas1      BishopAthanas1
Bishop_Sheen          Bishop_Sheen
                        ...       
settlerslament      settlerslament
sistervpaul_          sistervpaul_
staidansheritag    staidansheritag
stlaurencecha        stlaurencecha
timotheeology        timotheeology
Name: User, Length: 64, dtype: object

In [140]:
'''
More efficient line chart data generation
'''

'''
Desired output

Date	Sentiment	Count
0	2022-06-10 00:00:00	Negative	0.2
1	2022-06-10 00:00:00	Neutral	0.666667
2	2022-06-10 00:00:00	Positive	0.133333
3	2022-06-14 00:00:00	Negative	0.24
4	2022-06-14 00:00:00	Neutral	0.52
...	...	...	...
64	2022-07-07 00:00:00	Neutral	0.59
65	2022-07-07 00:00:00	Positive	0.14
66	2022-07-08 00:00:00	Negative	0.29
67	2022-07-08 00:00:00	Neutral	0.53
68	2022-07-08 00:00:00	Positive	0.18

'''

data = pd.read_csv('time_tweet.csv')
data['Time'] = pd.to_datetime(data['Time'])
pt_data = pd.pivot_table(data, values='User', index='Time', columns='Sentiment', aggfunc='count')
pt_data['Total'] = pt_data[['Negative','Neutral', 'Positive']].sum(axis=1)
pt_data['Negative'] = pt_data['Negative'] / pt_data['Total']
pt_data['Neutral'] = pt_data['Neutral'] / pt_data['Total']
pt_data['Positive'] = pt_data['Positive'] / pt_data['Total']
pt_data

KeyError: 'Time'

In [142]:
pd.options.plotting.backend = 'plotly'
pt_data.plot(y=['Negative', 'Neutral', 'Positive'])

In [135]:
def get_time_line_data(df):
    new_data = pd.DataFrame(columns=['Date', 'Sentiment', 'Count'])
    neg = df[df['Sentiment'] == 'Negative']
    neu = df[df['Sentiment'] == 'Neutral']
    pos = df[df['Sentiment'] == 'Positive']
    for frame, sentiment in zip([neg, neu, pos], ['Negative', 'Neutral', 'Positive']):
        frame = frame.groupby('Time').count()
        for i, date in enumerate(frame.index):
            row = pd.DataFrame([{'Date': date, 'Sentiment': sentiment,
                                'Count': frame.loc[date]['User']}])
            new_data = pd.concat([new_data, row], ignore_index=True)
    return new_data

d = get_time_line_data(data)

norm_data = pd.DataFrame(columns=['Date', 'Sentiment', 'Count'])
# Calculate percentages for each date
for date in d['Date'].unique():
    temp = d[d['Date'] == date]
    sum = temp['Count'].sum()
    for i, row in temp.iterrows():
        norm_row = row
        norm_row.loc['Count'] = row.loc['Count'] / sum
        norm_row = pd.DataFrame(
            [{'Date': norm_row.loc['Date'], 'Sentiment': norm_row.loc['Sentiment'], 'Count': norm_row.loc['Count']}])
        norm_data = pd.concat([norm_data, norm_row], ignore_index=True)
print(d)
norm_data

                   Date Sentiment Count
0   2022-06-10 00:00:00  Negative     6
1   2022-06-14 00:00:00  Negative    24
2   2022-06-15 00:00:00  Negative    28
3   2022-06-16 00:00:00  Negative    21
4   2022-06-17 00:00:00  Negative    38
..                  ...       ...   ...
64  2022-07-04 00:00:00  Positive    14
65  2022-07-05 00:00:00  Positive    18
66  2022-07-06 00:00:00  Positive    17
67  2022-07-07 00:00:00  Positive    14
68  2022-07-08 00:00:00  Positive    18

[69 rows x 3 columns]


,Date,Sentiment,Count
0,2022-06-10 00:00:00,Negative,0.2
1,2022-06-10 00:00:00,Neutral,0.666667
2,2022-06-10 00:00:00,Positive,0.133333
3,2022-06-14 00:00:00,Negative,0.24
4,2022-06-14 00:00:00,Neutral,0.52
...,...,...,...
64,2022-07-07 00:00:00,Neutral,0.59
65,2022-07-07 00:00:00,Positive,0.14
66,2022-07-08 00:00:00,Negative,0.29
67,2022-07-08 00:00:00,Neutral,0.53
